In [11]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca
from decision_vars import DecisionVarSet


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Test decision vars
from decision_vars import DecisionVarSet
dvs = DecisionVarSet(attrs = ['lb'])
dvs.add_vars({'x':[1.0, 2.0]}, lb={})
print(dvs)
dvs2 = DecisionVarSet(attrs = ['lb'])
dvs2.add_vars(inits={'y':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
d = {'test':1}
d.update(dvs2)
print(d)

***** Decision Vars *****
Attributes: ['sym', 'init', 'lb']
Vars: 
  x: [x_0, x_1], shape: (2, 1)

[-inf, -inf, 3]
{'x': array([0.1, 0.3]), 'y': array(0.5)}
{'test': 1, 'y': SX(y)}


In [18]:
from robot import *
rob = Robot('config/franka.urdf')

Building robot model from config/franka.urdf with TCP fr3_link8


In [19]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [10, 10, 10],
        'rest':[0, 0, 0.5]}
c = Contact(pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
c.F_fn(ca.DM([0.5, 0.3, -0.5]), ca.DM.ones(3))

c2 = Contact(pars, ['stiff'], {'cov':{}, 'noise':{}})
print(c2.F_fn(ca.DM([0.5, 0.3, -0.5]), ca.DM.ones(3), ca.DM([10, 0, 0])))

[-5, -0, 0]


In [21]:
from robot import Robot
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [10, 10, 10],
        'rest':[0, 0, 0.5]}
c2 = Contact(pars, ['stiff'], {'cov':{}, 'noise':{}})
rob = Robot('config/franka.urdf', subsys = [c2])
rob.build_disc_dyn(0.1)

Building robot model from config/franka.urdf with TCP fr3_link8
  with 1 subsys


Function(disc_dyn:(xi[17],tau_input[7])->(xi_next[17]) SXFunction)

In [61]:
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = ['pos'])
for name, model in models.items():
    step = model.build_disc_dyn(step_size=0.1)
    n_in = step.size_in(0)
    res = step(np.zeros(n_in), np.zeros(7))
    res_dict = model.get_statedict(res.full())
    print(f"{name} has step fn of {step}")
    print(f" which evals to {res_dict}")


Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1', 'contact_2']
plane has step fn of disc_dyn:(xi[17],tau_input[7])->(xi_next[17]) SXFunction
 which evals to {'q': array([ 8.93922588e-02,  5.06102852e+00,  8.92076159e-02,  1.02755344e+00,
        7.15815910e-02, -8.97063172e+00, -5.18415854e-03]), 'dq': array([ 8.93922588e-01,  5.06102852e+01,  8.92076159e-01,  1.02755344e+01,
        7.15815910e-01, -8.97063172e+01, -5.18415854e-02]), 'contact_1/pos': array([0., 0., 0.]), 'p': DM([-0.474956, -0.0876402, 0.0814489]), 'R': DM(
[[0.908743, 0.0785623, -0.409895], 
 [0.0144821, -0.987468, -0.157155], 
 [-0.417105, 0.136878, -0.898492]]), 'contact_1/F': DM([0, 0, 5887.4]), 'contact_1/disp': DM([-0.474956, -0.0876402, 0.0814489]), 'contact_1/n': DM([0, 0, 1]), 'contact_1/x': DM([-0.474956, -0.0876402, 0.0814489])}
hinge has step fn of disc_dyn:(xi[20],tau_input